# Intro
This ipython notebook is created to use Twitter API to collect data of interest. The first part is obtaining user id from the user name, the second part is collecting tweets from the users using streaming API, and the third part is collecting past 100 tweets from the users such that the activity level of the users can be estimated. The actual analysis using these data is done in the notebook 
twitter_NLP_insult_GloVe_RNN.ipynb.

In [1]:
try:
    import json
except ImportError:
    import simplejson as json

# Import the necessary methods from "twitter" library
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

In [11]:
import pandas as pd
# Obtain list of 50 most followed people worldwide from wikipedia
import wikipedia as wp
 
#Get the html source
#html = wp.page("List of most-followed Twitter accounts").html().encode("UTF-8")
#df = pd.read_html(html)[0]
#df.to_csv('twitter_list_of_influencers.csv',header=0,index=False)
df = pd.read_csv('twitter_list_of_influencers.csv')
# remove outlier
df = df.drop(df.index[17])
user_list = list(df['Account name'])[:]
# remove '@' infront of the screen names and save the list as user_list
user_list = [a[1:] for a in user_list]

In [12]:
print(df.head(20))

    Rank  Change (monthly)    Account name                Owner  \
0    1.0               NaN      @katyperry           Katy Perry   
1    2.0               NaN   @justinbieber        Justin Bieber   
2    3.0               NaN    @BarackObama         Barack Obama   
3    4.0               NaN        @rihanna              Rihanna   
4    5.0               NaN  @taylorswift13         Taylor Swift   
5    6.0               NaN       @ladygaga            Lady Gaga   
6    7.0               NaN   @TheEllenShow      Ellen DeGeneres   
7    8.0               NaN      @Cristiano    Cristiano Ronaldo   
8    9.0               NaN        @YouTube              YouTube   
9   10.0               NaN    @jtimberlake    Justin Timberlake   
10  11.0               NaN        @twitter              Twitter   
11  12.0               NaN  @KimKardashian  Kim Kardashian West   
12  13.0               NaN  @britneyspears       Britney Spears   
13  14.0               NaN   @ArianaGrande        Ariana Grand

In [13]:
# Variables that contains the user credentials to access Twitter API 
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

t = Twitter(auth=oauth)
id_list = t.users.lookup(screen_name=','.join(user_list))

In [16]:
user_id = [id_list[x]['id_str'] for x in range(49)]
#user_id = [id_list[x]['id_str'] for x in range(50)]

print(user_id)

['21447363', '27260086', '813286', '79293791', '17919972', '14230524', '15846407', '155659213', '10228272', '26565946', '783214', '25365536', '16409683', '34507480', '21111883', '23375688', '428333', '44409004', '15485441', '50393960', '85603854', '18839785', '100220864', '19397785', '807095', '23083404', '268414482', '759251', '105119490', '158487331', '180505807', '5402612', '27195114', '101311381', '26257166', '23151437', '145125358', '116362700', '2557521', '20322929', '132385468', '84279963', '28706024', '158314798', '181561712', '209708391', '35094637', '14872237', '60865434']


In [18]:
user_info = pd.DataFrame({"user_name":user_list,"user_id":user_id,"activity":df['Activity'],"country":df['Country'],"followers":df['Followers (millions)']})
user_info.head(20)
#user_info.to_csv("user_info.csv")

,activity,country,followers,user_id,user_name
0,Musician,USA,110,21447363,katyperry
1,Musician,CAN,107,27260086,justinbieber
2,Former U.S. President,USA,103,813286,BarackObama
3,Musician,BAR,89,79293791,rihanna
4,Musician,USA,86,17919972,taylorswift13
5,Musician,USA,79,14230524,ladygaga
6,Comedian,USA,78,15846407,TheEllenShow
7,Footballer,POR,74,155659213,Cristiano
8,Online video platform,USA,72,10228272,YouTube
9,Musician and actor,USA,66,26565946,jtimberlake


In [38]:
# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)
iterator = twitter_stream.statuses.filter(lang='en',follow=','.join(user_id))

# Get a sample of the public data following through Twitter
# iterator = twitter_stream.statuses.sample()
# As a pilot survey we set it to stop after getting 1000 tweets. 
# You don't have to set it to stop, but can continue running 
# the Twitter API to collect data for days or even longer. 

tweets_filename = 'twitter_savereplies_nsamp40000woDT.json'
#tweets_file = open(tweets_filename, "r")
import os.path
# find out a way to filter replies only:
# save to json file
tweet_count = 40000
tweet_cnt = tweet_count
tweets = []
try:
    os.path.isfile(tweets_filename)
    with open(tweets_filename, 'r') as f:
        tweets = json.load(f) # readline only the first tweet/line
        print("file exists")
        if len(tweets) < tweet_cnt:
            raise DataError
        pass
except:
    tweet_count = tweet_cnt - len(tweets)
    print("file not complete, stream twitter for collecting further tweets")
    new_tweets = []
    for tweet in iterator:
    # select only "replies" to top 50 followed users
        try:
            if str(tweet['in_reply_to_user_id']) in user_id:
        #print(tweet['in_reply_to_user_id'])
                tweet_count -= 1
                new_tweets.append(tweet)
        except:
            continue
        if tweet_count <= 0:
            break 
    # Twitter Python Tool wraps the data returned by Twitter 
    # as a TwitterDictResponse object.
    # We convert it back to the JSON format to print/score
    # loads converts json format to python dictionary
    # dumps converts python dictionary to json format
    # The command below will do pretty printing for JSON data, try it out
    # print json.dumps(tweet, indent=4)
#        with open(tweets_filename, 'w') as outfile:
#            json.dump(tweets,outfile,indent=4)

#    tweets.update(new_tweets)
    tweets.extend(new_tweets)
    with open(tweets_filename, 'w') as f:
        json.dump(tweets, f)

file exists
file not complete, stream twitter for collecting further tweets


In [39]:
print(len(tweets))

40000


In [9]:
# obtain past 100 tweets from 50 most followed users (including replies and RTs : any kind of activity by the user), 
# such that we can estimate how active the user is. 
#time_filename = "past_100_tweets.json" 
#tweets = []
#t = Twitter(auth=oauth)
#for i in range(len(user_id)):
#    tweet = t.statuses.user_timeline(user_id = user_id[i],count=100,trim_user=True)
#    tweets.append(tweet)
#with open(time_filename, 'w') as f:
#    json.dump(tweets, f)
#id_list = t.users.lookup(screen_name=','.join(user_list))

In [10]:
#print(len(tweets))

50
